### Osnovni RNN model za klasifikacijo slik z implementacijo RNN-modela iz knjižnice Tensorflow.contrib.rnn
Druga implementacija modela ponavljajoče nevronske mreže za klasifikacijo ročno zapisanih števil. Tokrat po drugem [tutorialu](https://pythonprogramming.net/rnn-tensorflow-python-machine-learning-tutorial/?completed=/recurrent-neural-network-rnn-lstm-machine-learning-tutorial/). Gre za preprostejšo kodo z uporabo že implementiranega modela, ki ga najdemo v knjižnici tensorflow.contrib.rnn.

__Opomba:__ Če se ta model zaganja v jupyter notebooku, je po vsakem zagonu programa potrebno ponovno naložiti kernel. Težava je v implementaciji RNN-celice in njenih uteži.  
Ta težava se ne pojavi ob zagonu programa izven jupyter notebooka.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn 

# loading the dataset
mnist = input_data.read_data_sets("../.datasets/mnist/", one_hot = True)

# basic hyperparameters
hm_epochs = 10
n_classes = 10
batch_size = 128

chunk_size = 28
n_chunks = 28
rnn_size = 256

# input placeholders
x = tf.placeholder('float', [None, n_chunks, chunk_size])
y = tf.placeholder('float')


def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size, n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    # data reshaping necesairy for the lstm cell input
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)
    
    # lstm cell and the recurrent layer
    lstm_cell = rnn.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']

    return output


def train_neural_network(x):
    # the function that actualy trains the network

    prediction = recurrent_neural_network(x)
    
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({
                    x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), 
                    y:mnist.test.labels}))

train_neural_network(x)

Extracting ../.datasets/mnist/train-images-idx3-ubyte.gz
Extracting ../.datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ../.datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ../.datasets/mnist/t10k-labels-idx1-ubyte.gz
Epoch 0 completed out of 10 loss: 189.467611138
Epoch 1 completed out of 10 loss: 53.9761687387
Epoch 2 completed out of 10 loss: 36.8233460402
Epoch 3 completed out of 10 loss: 28.3558302084
Epoch 4 completed out of 10 loss: 22.6149060365
Epoch 5 completed out of 10 loss: 20.4138097726
Epoch 6 completed out of 10 loss: 15.8014660076
Epoch 7 completed out of 10 loss: 14.0971603648
Epoch 8 completed out of 10 loss: 12.6619624599
Epoch 9 completed out of 10 loss: 10.601148039
Accuracy: 0.9842


Kot je razvidno zgoraj, je rezultat tega modela primerljiv s prejšnjo daljšo implementacijo. Ker pa je ta implementacija krajša in preprostejša je seveda boljša.